### Stratégie pour Utiliser le fichie JSON avec GPT-4o en RAG
	1.	Indexer les questions et réponses dans une base vectorielle (FAISS, ChromaDB).
	2.	Lorsqu’un utilisateur pose une question, récupérer la réponse la plus pertinente.
	3.	Envoyer la réponse trouvée + la question originale à GPT-4o pour générer une réponse contextualisée.

### Préparer la Base de Données Vectorielle

On va transformer les instructions et inputs en vecteurs pour effectuer des recherches de similarité.

In [1]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Charger les données du fichier JSON
with open("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/faq.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extraire les instructions et inputs comme données à indexer
texts = [entry["instruction"] + " " + entry["input"] for entry in data]
responses = [entry["output"] for entry in data]

# Utiliser SentenceTransformer pour encoder les textes
model = SentenceTransformer("all-MiniLM-L6-v2",device='cpu')
embeddings = model.encode(texts)

# Créer l'index FAISS pour la recherche rapide
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Sauvegarder l'index et les réponses
faiss.write_index(index, "C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/faq.faiss")
with open("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/responses.json", "w", encoding="utf-8") as f:
    json.dump(responses, f)

print("Base de données vectorielle créée ✅")

c:\Users\redaf\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] La procédure spécifiée est introuvable
  warn(f"Failed to load image Python extension: {e}")
c:\Users\redaf\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\redaf\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


Base de données vectorielle créée ✅


### Rechercher la Réponse la Plus Pertinente

Quand un utilisateur pose une question, on va chercher la réponse la plus proche dans FAISS.

In [2]:
def search_faq(query):
    query_vector = model.encode([query])  # Encoder la question

    # Charger l'index et les réponses
    index = faiss.read_index("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/faq.faiss")
    with open("C:/Users/redaf/OneDrive/Bureau/Datathonclub/Datathon/data/responses.json", "r", encoding="utf-8") as f:
        responses = json.load(f)

    # Trouver la réponse la plus proche
    _, idx = index.search(np.array(query_vector), k=1)  
    return responses[idx[0][0]]  # Retourner la meilleure réponse

# Exemple de recherche
question = "Comment nettoyer mon fauteuil roulant électrique ?"
print(search_faq(question))

Essuyez les pièces à l'aide d'un chiffon ou d'une brosse souple, d'agents de nettoyage ménagers ordinaires et d'eau chaude. Rincez ensuite à l'eau chaude et essuyez soigneusement les pièces avec un chiffon sec.


In [20]:
import datetime

def check_maintenance():
    # Exemple : Vérifier si la dernière maintenance a été effectuée il y a plus de 30 jours
    last_maintenance_date = datetime.datetime(2024, 1, 31)  # Date de la dernière maintenance
    current_date = datetime.datetime.now()
    delta = current_date - last_maintenance_date

    if delta.days > 30:
        return "Rappel : Une maintenance est nécessaire. Il y a plus de 30 jours depuis la dernière maintenance."
    else:
        return "Votre fauteuil n'a pas besoin de maintenance, vous êtes à jour!"
check_maintenance()

'Rappel : Une maintenance est nécessaire. Il y a plus de 30 jours depuis la dernière maintenance.'

### Intégrer GPT-4o avec OpenAI

Une fois qu’on a récupéré la réponse la plus pertinente, on l’envoie à GPT-4o en contexte pour une réponse plus fluide.

In [ ]:
from openai import OpenAI

client = OpenAI()

def ask_gpt4o(question):
    # Vérifier la maintenance avant de répondre
    print(f"Check rapide de maintenance : \n{check_maintenance()}\n")

    # Cherche une réponse dans la base de connaissances
    retrieved_answer = search_faq(question)

    # Vérifie si la question mentionne un modèle de fauteuil roulant
    models = ["AVIVA RX40", "Quickie Q50 R", "Permobil F5", "Invacare TDX SP2","AVIVA FX40","Juvo","LYNX","M3 Corpus","Sango","Storm","TDX","AVIVA RX40 ULM"]  # Liste d'exemples
    model_mentioned = any(model in question for model in models)

    if not model_mentioned:
        return "Pourriez-vous préciser le modèle du fauteuil roulant concerné ?"

    # Construire le prompt avec le contexte
    prompt = f"""On est dans le contexte des fauteuils roulants.
Il faut fournir une réponse en tenant compte du modèle mentionné et de la base de connaissances :
    
Réponse de la base : {retrieved_answer}

Question : {question}

Réponds avec précision en tenant compte du contexte.
"""

    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1000
    )

    return response.choices[0].message.content

# Exemple d'utilisation
question = "Comment changer les roues Storm?"
print(ask_gpt4o(question))


Check rapide de maintenance : 
Rappel : Une maintenance est nécessaire. Il y a plus de 30 jours depuis la dernière maintenance.

Pour changer les roues du fauteuil roulant Storm4 Max, il est important de suivre les instructions spécifiques du fabricant pour garantir la sécurité et le bon fonctionnement de l'appareil. Voici une démarche générale qui peut être suivie, mais il est toujours recommandé de consulter le manuel d'utilisation du Storm4 Max ou de contacter un professionnel qualifié pour une assistance spécifique :

1. **Sécurité d'abord** : Assurez-vous que le fauteuil est éteint et stable, éventuellement en utilisant une surface plane et en engageant les freins.

2. **Retrait de la roue** :
   - Si la roue est fixée par un écrou, utilisez une clé appropriée pour dévisser l'écrou qui tient la roue en place. Conservez l'écrou et les rondelles si présentes pour la réinstallation.
   - Si le fauteuil est équipé d'un système de fixation rapide, comme un bouton-poussoir central, appu